In [ ]:
import re
import string
from shapely.geometry import Point, MultiPolygon, Polygon
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from datetime import datetime

In [ ]:
trash = pd.read_csv('../trash-haulers-otto/trash_hauler_report.csv')

In [ ]:
trash.head()

In [ ]:
trash.tail()

In [ ]:
trash.shape

In [ ]:
type(trash)

In [ ]:
trash.dtypes

In [ ]:
#Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

#After determining the total amount of damages, you can look at other questions:

In [ ]:
trash['Request '].value_counts()

In [ ]:
trash.columns

In [ ]:
trash

In [ ]:
ee_miss = trash.loc[~trash['Request '].isin(['Trash - Curbside/Alley Missed Pickup'])]
ee_miss

In [ ]:
request = trash.loc[trash ['Request '] == "Trash - Curbside/Alley Missed Pickup" ]
request

In [ ]:
pd.set_option("max_colwidth", None)
ee_miss

In [ ]:
ee_miss.loc[ee_miss['Description'].str.contains(r'\bmiss\B',na = False)]

In [ ]:
total_misses = pd.concat([ee_miss, request])
total_misses

In [ ]:
total_misses['Description'] = total_misses['Description'].str.upper()

In [ ]:
total_misses['Description']

In [ ]:
total_misses['Zip Code'] = total_misses['Zip Code'].fillna(0)
total_misses['Zip Code'] = total_misses['Zip Code'].astype(int)

In [ ]:
total_misses['Council District'] = total_misses['Council District'].fillna(0)
total_misses['Council District'] = total_misses['Council District'].astype(int)

In [ ]:
total_misses

In [ ]:
total_misses['Zip Code'].value_counts()

In [ ]:
total_misses['Incident Address'] = total_misses['Incident Address'].str.strip()


In [ ]:
total_misses['Incident Address'] = total_misses['Incident Address'].str.lower()
total_misses['Incident Address'] = total_misses['Incident Address'].str.title()
total_misses

In [ ]:
total_misses['Incident Address']  = total_misses['Incident Address'] .str.replace(r'\bSt\b', 'Street', regex=True)
total_misses['Incident Address']  = total_misses['Incident Address'] .str.replace(r'\bAve\b', 'Avenue', regex=True)
total_misses['Incident Address']  = total_misses['Incident Address'] .str.replace(r'\bDr\b', 'Drive', regex=True)
total_misses

In [ ]:
total_misses['Incident Address'].fillna('Unknown', inplace=True) 


In [ ]:
total_misses.dropna(how='all')

In [ ]:
total_misses['Trash Hauler'] = total_misses['Trash Hauler'].str.lower()
total_misses['Trash Hauler'] = total_misses['Trash Hauler'].str.title()
total_misses

In [ ]:
remove = 'Nashville Tn'
total_misses['Incident Address'] = total_misses['Incident Address'].str.replace(remove, " ")


In [ ]:
total_misses['Incident Address'] = total_misses['Incident Address'].str.split(',', expand = True)[0]

In [ ]:
total_misses.drop_duplicates()

In [ ]:
total_misses['Date Opened'] = pd.to_datetime(total_misses['Date Opened'])
total_misses

In [ ]:
total_misses['Date Opened'].value_counts()

In [ ]:
#* What other types of complaints are there?

In [ ]:
trash['Request '].value_counts()

In [ ]:
address_count = total_misses['Incident Address'].value_counts()


In [ ]:
total_misses = total_misses.reset_index( drop = True)

In [ ]:
total_misses = total_misses.sort_values('Date Opened')


In [ ]:
total_misses

In [ ]:
total_misses.value_counts('Incident Address')

In [ ]:
tm = total_misses.groupby('Incident Address').rolling('182D', on = 'Date Opened').count('Request Number')


In [ ]:
tm2 = tm.loc[tm['Request Number'] >= 3].reset_index()
tm2

In [ ]:
tm2.value_counts('Incident Address')

In [ ]:
tm2 = tm2.drop(columns = ['Council District','State Plan X', 'State Plan Y', 'Zip Code'])


In [ ]:
tm3 = tm2.merge(total_misses[['Incident Address', 'Zip Code','Trash Hauler','Trash Route']], on='Incident Address', how='left')

In [ ]:
tm3.drop_duplicates()

In [ ]:
tm3 ['fine'] = (tm3['Request Number'] - 2) * 1500

In [ ]:
tm4 = tm3.drop_duplicates(['Incident Address','Date Opened']).groupby('Incident Address')['fine'].sum()

In [ ]:
tm4.sum()

In [ ]:
#* Make a heat map that shows the most total missed pick ups and another that shows the total fines, each by zip code.

In [ ]:
zipcodes = gpd.read_file('../geospatial-SarahLightner/data/zipcodes.geojson')

In [ ]:
zipcodes.dtypes

In [ ]:
joined_gdf = gpd.sjoin(tm4, zipcodes, how="inner", op="intersects")